# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [219]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [220]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# Tokenize and Stemming


In [221]:
X_data = mbti['posts'].ravel()
y_data = mbti['type'].ravel()

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [222]:
import nltk
import numpy as np
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils



In [224]:
ps=PorterStemmer()

stop_words = set(stopwords.words('english'))

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
stemData = np.array(stemData)


stemming 시작
stemming 완료


In [240]:
max_features = 10000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(stemData)
X = tokenizer.texts_to_sequences(stemData)

max_seq_length = np.max([len(text) for text in stemData])
print(max_seq_length)
X = pad_sequences(X)

print(nb_features)
print(X.shape)
print(X)


950
10000
(8675, 883)
[[   0    0    0 ...   60   63   22]
 [   0    0    0 ...  293  202    6]
 [   0    0    0 ...   33  559  591]
 ...
 [   0    0    0 ...  375   90  148]
 [   0    0    0 ... 1827  334 1455]
 [   0    0    0 ...  126   45   49]]


In [201]:
b_Pers = {'I':0, 'E':1, 'N':0, 'S':1, 'F':0, 'T':1, 'J':0, 'P':1}
b_Pers_list = [{0:'I', 1:'E'}, {0:'N', 1:'S'}, {0:'F', 1:'T'}, {0:'J', 1:'P'}]
y = np.array(y_data)

def translate_personality(personality):
    # transform mbti to binary vector
    
    return [b_Pers[l] for l in personality]

def translate_back(personality):
    # transform binary vector to mbti personality
    
    s = ""
    for i, l in enumerate(personality):
        s += b_Pers_list[i][l]
    return s
y = np.array([translate_personality(p) for p in y])

print(y)
print(y.shape)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]]
(8675, 4)


# Garbage Collection 1

In [162]:
import gc

'''del [[mbti]]

del sentence

del [[dataCombine]]

del [condition]

for ownsentence in allData:
    del ownsentence
del allData

for tokenData in tokenizedData:   # delete 'stemData'
    for data in tokenData:
        del data
    del tokenData
del tokenizedData

for clean_sentence in normalized_text:
    del clean_sentence
del normalized_text

del X_data

del tokenData
del tempData

del sentence

gc.collect()'''

"del [[mbti]]\n\ndel sentence\n\ndel [[dataCombine]]\n\ndel [condition]\n\nfor ownsentence in allData:\n    del ownsentence\ndel allData\n\nfor tokenData in tokenizedData:   # delete 'stemData'\n    for data in tokenData:\n        del data\n    del tokenData\ndel tokenizedData\n\nfor clean_sentence in normalized_text:\n    del clean_sentence\ndel normalized_text\n\ndel X_data\n\ndel tokenData\ndel tempData\n\ndel sentence\n\ngc.collect()"

In [226]:
import tensorflow as tf
from keras.utils import np_utils
from tensorflow import keras

from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [245]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)

#n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

print(X_train.shape)
print(y_train.shape)

X_train = np.expand_dims(X_train, 2)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)
print(X_train.shape[1])

model = Sequential()
model.add(Embedding(max_features, 128))
#model.add(Embedding(max_features, 128, max_seq_length))
model.add(LSTM(128, batch_input_shape=(None, X_train.shape[1])))
model.add(Dense(16, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split = 0.2)

(6072, 883)
(6072, 4)
(6072, 883, 1)
883
Epoch 1/5


ValueError: in user code:

    C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(data)
    C:\Python38\lib\site-packages\tens

In [ ]:
model.save('.\\models\\LSTM_undersample.h5')

In [ ]:
model.summary()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=32)